1. Build and train a keras sequential model to classify digits form the mnist dataset. The model
must have a hidden dense layer of 128 neurons with a relu activation function.
2. Now build and train a keras functional model for the same problem. The model must have
a hidden dense layer of 128 neurons with an activation function defined by the following
function:
activation(x) = ( 0 if x < 0, 
sin(x −π/2) + 1 if 0 ≤ x ≤ π/2,
x −π/2 + 1 otherwise )
3. Repeat the previous questions but now using PyTorch.

## 1. Build and train a keras sequential model to classify digits form the mnist dataset. The model must have a hidden dense layer of 128 neurons with a relu activation function.

In [7]:
# Importa las bibliotecas necesarias
from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.utils import to_categorical

# Carga el conjunto de datos MNIST
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

# Normaliza los datos de entrada
train_images = train_images.reshape((60000, 28 * 28))
train_images = train_images.astype('float32') / 255

test_images = test_images.reshape((10000, 28 * 28))
test_images = test_images.astype('float32') / 255

# Convierte las etiquetas a categorías
train_labels = to_categorical(train_labels)
test_labels = to_categorical(test_labels)

# Construye el modelo secuencial
model = Sequential()
model.add(Dense(128, activation='relu', input_shape=(28 * 28,)))
model.add(Dense(10, activation='softmax'))

# Compila el modelo
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Entrena el modelo
model.fit(train_images, train_labels, epochs=5, batch_size=128)

# Evalúa el modelo
test_loss, test_acc = model.evaluate(test_images, test_labels)
print('Test accuracy:', test_acc)


Epoch 1/5


AttributeError: module 'keras.src.backend' has no attribute 'convert_to_numpy'

In [8]:
import tensorflow as tf
import keras
print(tf.__version__)
print(keras.__version__)

2.16.1
3.3.2


In [9]:
# Importa las bibliotecas necesarias
from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.utils import to_categorical

# Carga el conjunto de datos MNIST
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

# Normaliza los datos de entrada
train_images = train_images.reshape((60000, 28 * 28))
train_images = train_images.astype('float32') / 255

test_images = test_images.reshape((10000, 28 * 28))
test_images = test_images.astype('float32') / 255

# Convierte las etiquetas a categorías
train_labels = to_categorical(train_labels)
test_labels = to_categorical(test_labels)

# Construye el modelo secuencial
model = Sequential()
model.add(Dense(128, activation='relu', input_shape=(28 * 28,)))
model.add(Dense(10, activation='softmax'))

# Compila el modelo
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Entrena el modelo
model.fit(train_images, train_labels, epochs=5, batch_size=128)

# Evalúa el modelo
test_loss, test_acc = model.evaluate(test_images, test_labels)
print('Test accuracy:', test_acc)

Epoch 1/5


AttributeError: module 'keras.src.backend' has no attribute 'convert_to_numpy'